In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import tensorflow as tf

In [2]:
DATA_PATH = "C:\\Users\\shota\\OneDrive\\ドキュメント\\Data\\Kaggle\\kc_Predict_Future_Sales\\"

basic_data = pd.read_csv(DATA_PATH + 'sales_train.csv')
basic_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
#basic_data.set_index('date', inplace=True)
basic_data['Year'] = basic_data['date'].apply(lambda x: int(x.split('.')[2]))
basic_data['Month'] = basic_data['date'].apply(lambda x: int(x.split('.')[1]))
basic_data['Day'] = basic_data['date'].apply(lambda x: int(x.split('.')[0]))
basic_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,Day
0,02.01.2013,0,59,22154,999.00,1.0,2013,1,2
1,03.01.2013,0,25,2552,899.00,1.0,2013,1,3
2,05.01.2013,0,25,2552,899.00,-1.0,2013,1,5
3,06.01.2013,0,25,2554,1709.05,1.0,2013,1,6
4,15.01.2013,0,25,2555,1099.00,1.0,2013,1,15


In [4]:
#basic_data['date'] = pd.to_datetime(basic_data['date'], format='%d.%m.%Y')   データタイムオブジェクトはsplitを使えない
fixed_numbers = basic_data[['Year','Month', 'shop_id','item_id','item_price']]
basic_data.drop(['date_block_num', 'item_price'], 1, inplace=True)

In [5]:
fixed_numbers.drop_duplicates(['Year','Month', 'shop_id','item_id'],ignore_index=True,keep='last', inplace=True)  #重複する行を削除
#keep='last'で重複した行の最後のほうを残す(今回は新しいほうを残すということ)

<ipython-input-5-00179258536c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_numbers.drop_duplicates(['Year','Month', 'shop_id','item_id'],ignore_index=True,keep='last', inplace=True)  #重複する行を削除


In [6]:
fixed_numbers

,Year,Month,shop_id,item_id,item_price
0,2013,1,59,22154,999.00
1,2013,1,25,2552,899.00
2,2013,1,25,2554,1709.05
3,2013,1,25,2555,1099.00
4,2013,1,25,2564,349.00
...,...,...,...,...,...
1609119,2015,10,25,7384,749.00
1609120,2015,10,25,7409,299.00
1609121,2015,10,25,7459,349.00
1609122,2015,10,25,7440,299.00


In [7]:
#basic_data['2013-01'].groupby(['shop_id','item_id']).count().rename(columns={'item_cnt_day':'item_cnt_month'})

In [8]:
basic_data.columns.values

array(['date', 'shop_id', 'item_id', 'item_cnt_day', 'Year', 'Month',
       'Day'], dtype=object)

In [9]:
basic_data = basic_data[['Year', 'Month', 'Day', 'shop_id', 'item_id', 'item_cnt_day']]
basic_data.head()

,Year,Month,Day,shop_id,item_id,item_cnt_day
0,2013,1,2,59,22154,1.0
1,2013,1,3,25,2552,1.0
2,2013,1,5,25,2552,-1.0
3,2013,1,6,25,2554,1.0
4,2013,1,15,25,2555,1.0


In [10]:
basic_data2 = basic_data.copy()

In [11]:
basic_data2.drop(['Day'], 1, inplace=True)

In [12]:
data_ordered = basic_data2.groupby(['Year', 'Month', 'shop_id', 'item_id']).count()

In [13]:
data_ordered.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [14]:
data_ordered.head()

item_cnt_month
Year Month shop_id item_id                
2013 1     0       32                    4
                   33                    3
                   35                    1
                   43                    1
                   51                    2

In [15]:
train_data = data_ordered.copy()

In [16]:
train_data.reset_index(inplace=True)    #MultiIndexをコラムに戻すことができる

In [17]:
train_data

,Year,Month,shop_id,item_id,item_cnt_month
0,2013,1,0,32,4
1,2013,1,0,33,3
2,2013,1,0,35,1
3,2013,1,0,43,1
4,2013,1,0,51,2
...,...,...,...,...,...
1609119,2015,10,59,22087,3
1609120,2015,10,59,22088,2
1609121,2015,10,59,22091,1
1609122,2015,10,59,22100,1


In [18]:
train_data = pd.merge(train_data, fixed_numbers, on=['Year','Month','shop_id', 'item_id'], how='inner')

In [19]:
train_data

,Year,Month,shop_id,item_id,item_cnt_month,item_price
0,2013,1,0,32,4,221.0
1,2013,1,0,33,3,347.0
2,2013,1,0,35,1,247.0
3,2013,1,0,43,1,221.0
4,2013,1,0,51,2,127.0
...,...,...,...,...,...,...
1609119,2015,10,59,22087,3,119.0
1609120,2015,10,59,22088,2,119.0
1609121,2015,10,59,22091,1,179.0
1609122,2015,10,59,22100,1,629.0


In [20]:
train_data_inputs = train_data.drop(['item_cnt_month', 'item_price'], axis=1).values

In [21]:
train_data_inputs

array([[ 2013,     1,     0,    32],
       [ 2013,     1,     0,    33],
       [ 2013,     1,     0,    35],
       ...,
       [ 2015,    10,    59, 22091],
       [ 2015,    10,    59, 22100],
       [ 2015,    10,    59, 22102]], dtype=int64)

In [22]:
train_data_targets = train_data[['item_cnt_month']].values

In [23]:
#train_data.index

In [24]:
#train_data.values.shape, train_data.index.values.shape

In [25]:
#train_data.index.values.reshape(1609124,1)

In [26]:
#train_data_inputs = train_data.index.values.reshape(len(train_data),1)

In [27]:
test_data_s = pd.read_csv(DATA_PATH + 'test.csv')
test_data_s.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [28]:
test_data_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


In [29]:
test_data_s['Year'] = 2015
test_data_s['Month'] = 11

In [30]:
test_data_s.head()

,ID,shop_id,item_id,Year,Month
0,0,5,5037,2015,11
1,1,5,5320,2015,11
2,2,5,5233,2015,11
3,3,5,5232,2015,11
4,4,5,5268,2015,11


In [31]:
#test_data_s = pd.merge(test_data_s, fixed_numbers, on=['shop_id', 'item_id', 'Year', 'Month'], how='inner')

In [32]:
test_data_s.columns.values

array(['ID', 'shop_id', 'item_id', 'Year', 'Month'], dtype=object)

In [33]:
test_data_inputs = test_data_s[['Year', 'Month', 'shop_id', 'item_id']]
test_data_inputs.head()

,Year,Month,shop_id,item_id
0,2015,11,5,5037
1,2015,11,5,5320
2,2015,11,5,5233
3,2015,11,5,5232
4,2015,11,5,5268


In [34]:
reg = LinearRegression()

In [35]:
reg.fit(train_data_inputs, train_data_targets)

LinearRegression()

In [36]:
reg.score(train_data_inputs, train_data_targets)

0.006131412189755769

In [37]:
reg.predict(train_data_inputs)

array([[1.98514391],
       [1.98512108],
       [1.98507544],
       ...,
       [1.60226685],
       [1.60206146],
       [1.60201581]])

In [38]:
train_data_targets

array([[4],
       [3],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

In [39]:
reg.predict(test_data_inputs)

array([[1.92420352],
       [1.91774511],
       [1.91973055],
       ...,
       [1.73899207],
       [1.6501946 ],
       [2.07647265]])

In [45]:
ans_data = test_data_s.copy()

In [46]:
ans_data['item_cnt_month'] = reg.predict(test_data_inputs)

In [47]:
ans_data = ans_data[['ID', 'item_cnt_month']]

In [49]:
ans_data.to_csv('kc_predict_sales_submission.csv', index=False)

In [ ]:
# tensorflow

In [60]:
df_train_inputs = train_data.drop(['item_cnt_month', 'item_price'], axis=1)
df_train_targets = train_data[['item_cnt_month']]

In [61]:
shuffled_indices = np.arange(df_train_inputs.shape[0])
np.random.shuffle(shuffled_indices)

train_shuffled_inputs = df_train_inputs.reindex(index=shuffled_indices)
train_shuffled_targets = df_train_targets.reindex(index=shuffled_indices)

In [64]:
train_shuffled_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609124 entries, 318170 to 282972
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   Year     1609124 non-null  int64
 1   Month    1609124 non-null  int64
 2   shop_id  1609124 non-null  int64
 3   item_id  1609124 non-null  int64
dtypes: int64(4)
memory usage: 61.4 MB


In [66]:
p_count = train_shuffled_inputs.shape[0]

train_p_count = round(p_count * 0.8)  #四捨五入して整数に
val_p_count = p_count - train_p_count 

train_p_inputs = train_shuffled_inputs[:train_p_count]
train_p_targets = train_shuffled_targets[:train_p_count]

val_p_inputs = train_shuffled_inputs[train_p_count:]
val_p_targets = train_shuffled_targets[train_p_count:]

In [67]:
np_train_inputs = train_p_inputs.values
np_train_targets = train_p_targets.values

np_val_inputs = val_p_inputs.values
np_val_targets = val_p_targets.values

In [74]:
np_train_inputs

array([[ 2013,     6,    28, 12795],
       [ 2013,     1,    46, 17392],
       [ 2014,    12,    27, 21872],
       ...,
       [ 2014,     6,    26, 18070],
       [ 2014,    11,    31, 17125],
       [ 2014,     9,    29,  2607]], dtype=int64)

In [79]:
inputs_size = 4
outputs_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(outputs_size, activation='linear')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])    #出力が確率ではないときは、損失関数にbinary_crossentropyを選択する

batch_size=100
max_epochs=100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(np_train_inputs,
          np_train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data = (np_val_inputs, np_val_targets),
          verbose=2
         )

Epoch 1/100
12873/12873 - 18s - loss: -1.2582e+01 - accuracy: 0.6691 - val_loss: -1.2537e+01 - val_accuracy: 0.6699
Epoch 2/100
12873/12873 - 17s - loss: -1.2582e+01 - accuracy: 0.6691 - val_loss: -1.2537e+01 - val_accuracy: 0.6699
Epoch 3/100
12873/12873 - 19s - loss: -1.2582e+01 - accuracy: 0.6691 - val_loss: -1.2537e+01 - val_accuracy: 0.6699


In [81]:
model.predict(test_data_inputs.values)   #失敗

array([[1758.128 ],
       [1858.5308],
       [1828.4147],
       ...,
       [5271.317 ],
       [6547.3496],
       [ 340.7222]], dtype=float32)

In [82]:
np_train_targets

array([[1],
       [1],
       [1],
       ...,
       [1],
       [2],
       [1]], dtype=int64)

In [ ]:
# シャッフルした訓練データで再度チャレンジ(sklearn)

In [83]:
reg.fit(train_shuffled_inputs, train_shuffled_targets)

LinearRegression()

In [89]:
reg.predict(test_data_inputs)

array([[1.92420352],
       [1.91774511],
       [1.91973055],
       ...,
       [1.73899207],
       [1.6501946 ],
       [2.07647265]])

In [88]:
#reg.predict(test_data_inputs).round().astype('int')    #四捨五入して、int化する

array([[2],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]])

In [106]:
np.floor(reg.predict(test_data_inputs)).astype('int')    #切り上げして、int化

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [2]])

In [111]:
ans_data2 = test_data_s.copy()

In [112]:
ans_data2.drop(['Year','Month'],1, inplace=True)

In [113]:
ans_data2['item_cnt_month'] = np.floor(reg.predict(test_data_inputs)).astype('int')

In [114]:
redict_sales_submission2.csvans_data2[['ID','item_cnt_month']].to_csv('kc_predict_sales_submission2.csv', index=False)

In [ ]:
#RandomForestRegressor で再チャレンジ   (決定木型の回帰)

In [117]:
from sklearn.ensemble import RandomForestRegressor

In [118]:
model = RandomForestRegressor()

In [133]:
train_shuffled_inputs.apply(preprocessing.scale)   #標準化をする例

,Year,Month,shop_id,item_id
318170,-1.019532,-0.044747,-0.290612,0.338844
50376,-1.019532,-1.491829,0.797856,1.075675
1219603,0.266919,1.691751,-0.351082,1.793753
520866,-1.019532,1.112918,-1.681432,-0.003685
1036652,0.266919,0.534085,-0.411553,0.665185
...,...,...,...,...
836965,0.266919,-0.912996,1.523502,0.878204
734268,0.266919,-1.491829,1.281620,1.137866
1296706,1.553370,-1.491829,1.281620,1.807056
190610,-1.019532,-0.623580,-1.560491,-1.244132


In [137]:
model.fit(train_shuffled_inputs.apply(preprocessing.scale), train_shuffled_targets)

<ipython-input-137-f5fb04c5833c>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_shuffled_inputs.apply(preprocessing.scale), train_shuffled_targets)


RandomForestRegressor()

In [138]:
model.predict(test_data_inputs.apply(preprocessing.scale))

array([1.48, 1.11, 1.28, ..., 1.22, 1.23, 1.13])

In [141]:
predict_targets = model.predict(test_data_inputs.apply(preprocessing.scale))

In [145]:
predict_targets.round()

array([1., 1., 1., ..., 1., 1., 1.])

In [146]:
ans_data3 = test_data_s.copy()

In [147]:
ans_data3.drop(['Year','Month'],1, inplace=True)

In [148]:
ans_data3['item_cnt_month'] = predict_targets.round()

In [149]:
ans_data3[['ID', 'item_cnt_month']].to_csv('kc_predict_sales_submission3.csv', index=False)

In [ ]:
# 標準化せずにもう一度行う

In [150]:
model.fit(train_shuffled_inputs, train_shuffled_targets)

<ipython-input-150-bfbcd0575ff3>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_shuffled_inputs, train_shuffled_targets)


RandomForestRegressor()

In [154]:
prediction = model.predict(test_data_inputs)

In [155]:
prediction

array([1.32, 1.94, 1.35, ..., 1.1 , 1.04, 1.03])

In [156]:
ans_data4 = test_data_s.copy()

In [157]:
ans_data4['item_cnt_month'] = prediction.round()

In [158]:
ans_data4[['ID', 'item_cnt_month']].to_csv('kc_predict_sales_submission4.csv', index=False)